<h1>Predicting House Prices</h1>
<div style="text-align: right"></div>


### Importing the required libraries

In [88]:
#importing libraries 
import numpy as np 
import pandas as pd 
pd.options.display.max_columns = 500
import seaborn as sns
sns.set_style('darkgrid')
import re
import pickle
import numpy as np
import pandas as pd
import sklearn
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import xgboost
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

def super_star(*n):
    m=n[0]
    if len(n)>1:
        rep=n[1]
        for i in (0,rep):
            print("*"*m,"\n")
            
    else:
        print("*"*m)  


**Reading the Test and Train Data into** **`DataFrame`**. 

In [89]:
train = pd.read_csv("train.csv") 
test = pd.read_csv("test.csv") 

___
<h3> Data Preprocessing</h3>
<a id="prep"></a>

Preparing the data to bring it to such a state so that the features of the data can be easily interpreted by the algorithm

***

**Overview of the data**
<a id="dims"></a>

In [90]:
#to know about the dimentions of the dataset
print("Shape of Train Data: "+str(train.shape)+"\nShape of Test Data: "+str(test.shape))


Shape of Train Data: (13320, 9)
Shape of Test Data: (1480, 9)


In [91]:
#Insights about Each Column

#-------------PRINTING THE COLUMN NAMES------------------------------------------------------------
super_star(100,2)
print("Column Names:")
super_star(12)
print( "\n",*train.columns.values,sep='\n')

#------------Applying .info to get an overview of the data-----------------------------------------
super_star(100,2)
print("Overview of the Train data: ")
super_star(27)
print("\n",train.info())# we see some values are missing from the column education and previous_year_rating


#-------------CHECKING THE NUMBER OF MISSING VALUES-----------------------------------------------
super_star(100,2)
print("Number of Missing values in each Columns:")
super_star(40)
print("\n", train.isnull().sum()) #number of null values in each column


super_star(100,2)
print("Percentage of Missing values in each Columns:")
super_star(40)
print("\n",(train.isnull().mean()*100))#percentage of null values in each column
super_star(100)

**************************************************************************************************** 

**************************************************************************************************** 

Column Names:
************


area_type
availability
location
size
society
total_sqft
bath
balcony
price
**************************************************************************************************** 

**************************************************************************************************** 

Overview of the Train data: 
***************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft   

___
### Observations:
- We can see that the train set has got  5502 misssing values in `society` and 609 misssing values in the `balcony` columns respectively.Which amounts to 41.3% of missing data in `society` column and 4.5% of missing data in the `balcony` column.
- We have 5 categorical columns in our dataset (` availability,location,size,area_type,society`).<br><br>

#### Inference:
1. Missing data in `balcony` column might represent that the residence has no balconies.<br>
2. Missing data in `society` column might be due to human error or Maybe all these house doesn't belong to any society.
3. `size` can be an important feature in determning the value.
4. `location` columns can help fill the features as there are variations in pricing w.r.t location
___


In [92]:
#train.drop(columns='society', axis=1,inplace=True)

#Insights about the values in the data

super_star(100)
print("Getting an overview about the categorical features:\n")
print(train.describe(include = 'object').transpose())#to get an overview of the categorical features
super_star(100,2)
print()
super_star(100)
print("Getting an overview about the non categorical features:")
train.describe().transpose()#to get the overview of the non categorical features

****************************************************************************************************
Getting an overview about the categorical features:

              count unique                   top   freq
area_type     13320      4  Super built-up  Area   8790
availability  13320     81         Ready To Move  10581
location      13319   1305            Whitefield    540
size          13304     31                 2 BHK   5199
society        7818   2688               GrrvaGr     80
total_sqft    13320   2117                  1200    843
**************************************************************************************************** 

**************************************************************************************************** 


****************************************************************************************************
Getting an overview about the non categorical features:


,count,mean,std,min,25%,50%,75%,max
bath,13247.0,2.692610,1.341458,1.0,2.0,2.0,3.0,40.0
balcony,12711.0,1.584376,0.817263,0.0,1.0,2.0,2.0,3.0
price,13320.0,112.565627,148.971674,8.0,50.0,72.0,120.0,3600.0


In [93]:
train.area_type.value_counts()

Super built-up  Area    8790
Built-up  Area          2418
Plot  Area              2025
Carpet  Area              87
Name: area_type, dtype: int64

___


<h3> Observations: </h3><br>
<h4>From the trends in the train data it can be derive that:</h4>

- Only one value in `location` column is null so we can drop it
- There are a total of  4 `area_type` in bangalore with most of the `Super built-up Area` type.
- There are 4 unique categories. So, replacing them with (0-4).<br>
#### The `availability` feature has three kind of values
- Ready to move
- Immediate Possession
- Dates (17-DEC,23-NOV...)<br><br>

***So we have classified all the date category into `others`, so now we have 3 categories under the column `availability` which are again replaced by (0-2) respectively:***
- Ready to move
- Immediate Possession
- others

**The `size` feature is also helpful indetermining the value of the residence and it contains categorical data too, so we need to replace all the categories to numeric data**

___

In [94]:
#---------------------------------col:area_type to numeric--------------------------------------------

def area_type_to_bin(df):
    if df == 'Super built-up  Area':#replaces the two categories 
        return 1
    elif df == 'Built-up  Area':
        return 2
    elif df == 'Plot  Area':
        return 3
    elif df == 'Carpet  Area':
        return 4
    else:              #any other category will get replaced by 2
        return 0


#---------------------------------col:availability to numeric--------------------------------------------
def availabilty_to_bin(df):
    if df == 'Ready To Move':#replaces the two categories `Ready to move` & `Immediate Possession` with 0 and 1
        return 1
    elif df == 'Immediate Possession':
        return 2
    else:              #any other category will get replaced by 0
        return 0
    
    
train['area_type'] = train.area_type.map(area_type_to_bin)    
train['availability'] = train.availability.apply(availabilty_to_bin) 

test['area_type'] = test.area_type.map(area_type_to_bin)    
test['availability'] = test.availability.apply(availabilty_to_bin) 




#-------------------------------------Encoding ------------------------------------------------------------------------------------
def encode(col_name):
    #using LabelEncoder() as there are multiple unique value
    le=LabelEncoder()
    #--checking for null and replaceing with 'other' value
    train[col_name] = train[col_name].fillna('Other') 
    test[col_name] = test[col_name].fillna('Other') 
    
    le.fit(train[col_name].astype('str').append(test[col_name].astype('str')))
    
    #applying on both test and train
    train[col_name] = le.transform(train[col_name].astype('str'))
    test[col_name] = le.transform(test[col_name].astype('str'))

col_name=['size','society','location']
for i in col_name:
    encode(i)

In [95]:
#---------VALUES UNDER THE TRAIN DATA-----------------
train.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,1,0,430,14,487,1056,2.0,1.0,39.07
1,3,1,325,20,2569,2600,5.0,3.0,120.00
2,2,1,1220,17,1528,1440,2.0,3.0,62.00
3,1,1,778,17,2303,1521,3.0,1.0,95.00
4,1,1,736,14,1528,1200,2.0,1.0,51.00


In [96]:
test.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,1,1,284,14,1876,1225,2.0,2.0,NaN
1,3,1,103,31,1528,2400,9.0,2.0,NaN
2,3,0,534,20,2030,1650,5.0,2.0,NaN
3,1,1,721,17,2817,1322,3.0,1.0,NaN
4,1,1,727,14,135,1161,2.0,1.0,NaN


___

## Mixed variable seperation

### Observation :
On looking at the values of each columns we see that the `total_sqft` column consist of Mixed Variables.

#### Inference:
<h3>The column `total_sqft` was supposed to have the area in square feet but there are some values in other metrics and some range values that can't be processed(e.g.:-188.89Sq. Yards','1469 - 1766','204Sq. Meter',).</h3>

_As all the given values are not measured in 'square feet' metric, we have to preprocess some of the values in the column._

**Some of them are also measured in:**<br>
- square meters
- square yards
- perch
- acres
- cents
- guntha
- grounds

**So, they need to be converted into square feet to make the data consistent.**

In [97]:
#list1=train.total_sqft
#train.total_sqft.astype('float64')
#df=train.total_sqft
#uniq=list(df[df.apply(lambda x:not x.isnumeric())])
#uniq

In [98]:
def convert_2_sqft(df):#this converts all type of metrics to sqft
    if len(df) == 1:
        
        try:
            return float(df[0])
        except:
            strings = ['Sq. Meter', 'Sq. Yards', 'Perch', 'Acres', 'Cents', 'Guntha', 'Grounds']
            split_list = re.split('(\d*.*\d)', df[0])[1:]  #splits on whitespace and stores the numeric and the string part
            area_val = float(split_list[0]) #numeric
            type_of_area = split_list[1] #string
            
            conversion(type_of_area,area_val)#this function takes metrics and return value in sq.ft. 
            
        
    else:
        return (float(df[0]) + float(df[1]))/2.0 #for values such as ('1469 - 1766') it will take the average in this range 
                                                            #(1469+1766)/2
    
def conversion(metric,area_val):
    
    if metric == 'Sq. Meter':
        metric_in_sqft = area_val * 10.7639
    elif metric == 'Sq. Yards':
        metric_in_sqft = area_val * 9.0
    elif metric == 'Perch':
        metric_in_sqft = area_val * 272.25
    elif metric == 'Acres':
        metric_in_sqft = area_val * 43560.0
    elif metric == 'Cents':
        metric_in_sqft = area_val * 435.61545
    elif metric == 'Guntha':
        metric_in_sqft = area_val * 1089.0
    elif metric == 'Grounds':
        metric_in_sqft = area_val * 2400.0
    return float(metric_in_sqft)


In [99]:
train['total_sqft'] = train.total_sqft.str.split('-').apply(convert_2_sqft)#splittin on '-' for values such as ('1469 - 1766')
test['total_sqft'] = test.total_sqft.str.split('-').apply(convert_2_sqft) 

In [100]:
train.head()
test.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,1,1,284,14,1876,1225.0,2.0,2.0,NaN
1,3,1,103,31,1528,2400.0,9.0,2.0,NaN
2,3,0,534,20,2030,1650.0,5.0,2.0,NaN
3,1,1,721,17,2817,1322.0,3.0,1.0,NaN
4,1,1,727,14,135,1161.0,2.0,1.0,NaN


___
### Handling Missing values:
<a id="miss"></a>
- filling missing values of  __`balcony`__ by grouping the rows based on location and taking the mean of the column 'balcony' in that location.
- filling missing values of  __`bath`__ by grouping grouping the rows based on location and taking the mean of the column 'bath' in that location.

In [101]:
#filling values in 'balcony' based on locationwise mean 
loc_balcony = train.groupby('location')['balcony'].transform(lambda x: x.fillna(x.mean()))
loc_balcony_ts = test.groupby('location')['balcony'].transform(lambda x: x.fillna(x.mean()))

#filling values in 'bath' based on locationwise mean 
loc_bath = train.groupby('location')['bath'].transform(lambda x: x.fillna(x.mean()))
loc_bath_ts = test.groupby('location')['bath'].transform(lambda x: x.fillna(x.mean()))

train['balcony'] = loc_balcony
train['bath'] = loc_bath

test['balcony'] = loc_balcony_ts
test['bath'] = loc_bath_ts


In [115]:
train.head().isnull().sum()

area_type       0
availability    0
location        0
size            0
society         0
total_sqft      0
bath            0
balcony         0
price           0
dtype: int64

____
### Separating feature set from target variable

In [112]:
#Splitting data into Feature set and Target 

#--------------Copying data from 'train' to 'X' except the target column 'is_promoted' to make the feature set------------------
X_train = train.drop(['price'], axis = 1)   #feature set
y_train = train['price']  # Target Column

print("Shape of X_train(Feature set): "+str(X_train.shape)+"\nShape of y_train(Target Column): "+str(y_train.shape))
X_train.head()

Shape of X_train(Feature set): (13320, 8)
Shape of y_train(Target Column): (13320,)


,area_type,availability,location,size,society,total_sqft,bath,balcony
0,1,0,430,14,487,1056.0,2.0,1.0
1,3,1,325,20,2569,2600.0,5.0,3.0
2,2,1,1220,17,1528,1440.0,2.0,3.0
3,1,1,778,17,2303,1521.0,3.0,1.0
4,1,1,736,14,1528,1200.0,2.0,1.0


In [113]:
test = test.drop(columns='price')
test.isna().sum()

X_test = test
X_test.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony
0,1,1,284,14,1876,1225.0,2.0,2.0
1,3,1,103,31,1528,2400.0,9.0,2.0
2,3,0,534,20,2030,1650.0,5.0,2.0
3,1,1,721,17,2817,1322.0,3.0,1.0
4,1,1,727,14,135,1161.0,2.0,1.0



## Modelling


In [130]:
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import RandomizedSearchCV
#lr = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,random_state=0)

#param = dict(C=uniform(loc=0, scale=4),
#clf = RandomizedSearchCV(logistic, distributions, random_state=0)
#search = clf.fit((X_train, y_train)
#search.best_params_

In [128]:
#random forest regressor

#model=RandomForestRegressor(n_estimators=1000, criterion='mse', max_depth=5,random_state=42)
#model.fit(X_train, y_train)

#y_pred=model.predict(X_test)
#generate_test_files(y_pred)

In [18]:
#catboost
#from catboost import CatBoostRegressor
#from sklearn.model_selection import RandomizedSearchCV
#model=CatBoostRegressor(random_seed=22, iterations=15000, depth=8, learning_rate=0.003, loss_function='RMSE')
#model.fit(X_train, y_train)

0:	learn: 151.4977786	total: 149ms	remaining: 37m 19s
1:	learn: 151.2776206	total: 155ms	remaining: 19m 19s
2:	learn: 151.0457245	total: 160ms	remaining: 13m 18s
3:	learn: 150.8082058	total: 164ms	remaining: 10m 16s
4:	learn: 150.5847605	total: 169ms	remaining: 8m 27s
5:	learn: 150.3516426	total: 174ms	remaining: 7m 14s
6:	learn: 150.1176220	total: 179ms	remaining: 6m 23s
7:	learn: 149.8859281	total: 184ms	remaining: 5m 44s
8:	learn: 149.6638688	total: 189ms	remaining: 5m 14s
9:	learn: 149.4272382	total: 194ms	remaining: 4m 50s
10:	learn: 149.2052438	total: 198ms	remaining: 4m 30s
11:	learn: 148.9854102	total: 203ms	remaining: 4m 13s
12:	learn: 148.7617987	total: 208ms	remaining: 4m
13:	learn: 148.5321039	total: 213ms	remaining: 3m 48s
14:	learn: 148.3012807	total: 218ms	remaining: 3m 38s
15:	learn: 148.0714853	total: 223ms	remaining: 3m 29s
16:	learn: 147.8399060	total: 228ms	remaining: 3m 21s
17:	learn: 147.6275017	total: 233ms	remaining: 3m 13s
18:	learn: 147.4088217	total: 238ms	re

165:	learn: 122.3529497	total: 1.03s	remaining: 1m 32s
166:	learn: 122.2277345	total: 1.04s	remaining: 1m 32s
167:	learn: 122.0993814	total: 1.04s	remaining: 1m 32s
168:	learn: 121.9635352	total: 1.05s	remaining: 1m 32s
169:	learn: 121.8328156	total: 1.06s	remaining: 1m 32s
170:	learn: 121.6978423	total: 1.06s	remaining: 1m 32s
171:	learn: 121.5689624	total: 1.07s	remaining: 1m 32s
172:	learn: 121.4360166	total: 1.08s	remaining: 1m 32s
173:	learn: 121.3009086	total: 1.08s	remaining: 1m 32s
174:	learn: 121.1804983	total: 1.09s	remaining: 1m 32s
175:	learn: 121.0560978	total: 1.09s	remaining: 1m 32s
176:	learn: 120.9172492	total: 1.1s	remaining: 1m 32s
177:	learn: 120.7854018	total: 1.11s	remaining: 1m 32s
178:	learn: 120.6643354	total: 1.11s	remaining: 1m 32s
179:	learn: 120.5290535	total: 1.12s	remaining: 1m 31s
180:	learn: 120.4013534	total: 1.12s	remaining: 1m 31s
181:	learn: 120.2828590	total: 1.13s	remaining: 1m 31s
182:	learn: 120.1569827	total: 1.13s	remaining: 1m 31s
183:	learn:

323:	learn: 105.9955346	total: 1.91s	remaining: 1m 26s
324:	learn: 105.9179017	total: 1.92s	remaining: 1m 26s
325:	learn: 105.8379561	total: 1.93s	remaining: 1m 26s
326:	learn: 105.7503190	total: 1.93s	remaining: 1m 26s
327:	learn: 105.6731682	total: 1.94s	remaining: 1m 26s
328:	learn: 105.5859318	total: 1.94s	remaining: 1m 26s
329:	learn: 105.5050310	total: 1.95s	remaining: 1m 26s
330:	learn: 105.4313377	total: 1.96s	remaining: 1m 26s
331:	learn: 105.3550047	total: 1.96s	remaining: 1m 26s
332:	learn: 105.2752390	total: 1.97s	remaining: 1m 26s
333:	learn: 105.1936075	total: 1.97s	remaining: 1m 26s
334:	learn: 105.1173080	total: 1.98s	remaining: 1m 26s
335:	learn: 105.0418635	total: 1.99s	remaining: 1m 26s
336:	learn: 104.9659609	total: 1.99s	remaining: 1m 26s
337:	learn: 104.8899732	total: 2s	remaining: 1m 26s
338:	learn: 104.8067518	total: 2s	remaining: 1m 26s
339:	learn: 104.7232978	total: 2.01s	remaining: 1m 26s
340:	learn: 104.6503248	total: 2.01s	remaining: 1m 26s
341:	learn: 104.

498:	learn: 95.0170179	total: 2.96s	remaining: 1m 25s
499:	learn: 94.9655790	total: 2.96s	remaining: 1m 25s
500:	learn: 94.9220988	total: 2.97s	remaining: 1m 25s
501:	learn: 94.8744168	total: 2.98s	remaining: 1m 25s
502:	learn: 94.8206378	total: 2.98s	remaining: 1m 25s
503:	learn: 94.7741713	total: 2.99s	remaining: 1m 25s
504:	learn: 94.7211608	total: 2.99s	remaining: 1m 25s
505:	learn: 94.6752785	total: 3s	remaining: 1m 25s
506:	learn: 94.6288276	total: 3.01s	remaining: 1m 25s
507:	learn: 94.5786389	total: 3.01s	remaining: 1m 25s
508:	learn: 94.5331084	total: 3.02s	remaining: 1m 25s
509:	learn: 94.4900433	total: 3.02s	remaining: 1m 25s
510:	learn: 94.4458518	total: 3.03s	remaining: 1m 25s
511:	learn: 94.4007693	total: 3.04s	remaining: 1m 25s
512:	learn: 94.3498415	total: 3.04s	remaining: 1m 25s
513:	learn: 94.3020645	total: 3.05s	remaining: 1m 25s
514:	learn: 94.2533896	total: 3.05s	remaining: 1m 25s
515:	learn: 94.1996905	total: 3.06s	remaining: 1m 25s
516:	learn: 94.1601739	total: 3

675:	learn: 87.9881153	total: 4s	remaining: 1m 24s
676:	learn: 87.9533798	total: 4.01s	remaining: 1m 24s
677:	learn: 87.9171056	total: 4.02s	remaining: 1m 24s
678:	learn: 87.8871492	total: 4.02s	remaining: 1m 24s
679:	learn: 87.8536031	total: 4.03s	remaining: 1m 24s
680:	learn: 87.8233649	total: 4.04s	remaining: 1m 24s
681:	learn: 87.7950940	total: 4.04s	remaining: 1m 24s
682:	learn: 87.7589606	total: 4.05s	remaining: 1m 24s
683:	learn: 87.7277015	total: 4.05s	remaining: 1m 24s
684:	learn: 87.6968512	total: 4.06s	remaining: 1m 24s
685:	learn: 87.6582217	total: 4.07s	remaining: 1m 24s
686:	learn: 87.6292403	total: 4.08s	remaining: 1m 24s
687:	learn: 87.6021369	total: 4.08s	remaining: 1m 24s
688:	learn: 87.5718652	total: 4.09s	remaining: 1m 24s
689:	learn: 87.5428884	total: 4.09s	remaining: 1m 24s
690:	learn: 87.5175347	total: 4.1s	remaining: 1m 24s
691:	learn: 87.4881870	total: 4.11s	remaining: 1m 25s
692:	learn: 87.4501104	total: 4.12s	remaining: 1m 25s
693:	learn: 87.4130618	total: 4.

...

14324:	learn: 38.8609612	total: 1m 17s	remaining: 3.64s
14325:	learn: 38.8600412	total: 1m 17s	remaining: 3.64s
14326:	learn: 38.8590994	total: 1m 17s	remaining: 3.63s
14327:	learn: 38.8580793	total: 1m 17s	remaining: 3.63s
14328:	learn: 38.8566713	total: 1m 17s	remaining: 3.62s
14329:	learn: 38.8552674	total: 1m 17s	remaining: 3.61s
14330:	learn: 38.8539977	total: 1m 17s	remaining: 3.61s
14331:	learn: 38.8532501	total: 1m 17s	remaining: 3.6s
14332:	learn: 38.8521429	total: 1m 17s	remaining: 3.6s
14333:	learn: 38.8509588	total: 1m 17s	remaining: 3.59s
14334:	learn: 38.8503439	total: 1m 17s	remaining: 3.59s
14335:	learn: 38.8485898	total: 1m 17s	remaining: 3.58s
14336:	learn: 38.8474398	total: 1m 17s	remaining: 3.58s
14337:	learn: 38.8462243	total: 1m 17s	remaining: 3.57s
14338:	learn: 38.8451391	total: 1m 17s	remaining: 3.57s
14339:	learn: 38.8440138	total: 1m 17s	remaining: 3.56s
14340:	learn: 38.8422181	total: 1m 17s	remaining: 3.56s
14341:	learn: 38.8419072	total: 1m 17s	remaining: 

14490:	learn: 38.6729050	total: 1m 18s	remaining: 2.75s
14491:	learn: 38.6720682	total: 1m 18s	remaining: 2.74s
14492:	learn: 38.6711845	total: 1m 18s	remaining: 2.73s
14493:	learn: 38.6699201	total: 1m 18s	remaining: 2.73s
14494:	learn: 38.6680587	total: 1m 18s	remaining: 2.72s
14495:	learn: 38.6661852	total: 1m 18s	remaining: 2.72s
14496:	learn: 38.6647107	total: 1m 18s	remaining: 2.71s
14497:	learn: 38.6636763	total: 1m 18s	remaining: 2.71s
14498:	learn: 38.6626055	total: 1m 18s	remaining: 2.7s
14499:	learn: 38.6615446	total: 1m 18s	remaining: 2.7s
14500:	learn: 38.6600952	total: 1m 18s	remaining: 2.69s
14501:	learn: 38.6587871	total: 1m 18s	remaining: 2.69s
14502:	learn: 38.6578338	total: 1m 18s	remaining: 2.68s
14503:	learn: 38.6569140	total: 1m 18s	remaining: 2.67s
14504:	learn: 38.6557851	total: 1m 18s	remaining: 2.67s
14505:	learn: 38.6553300	total: 1m 18s	remaining: 2.66s
14506:	learn: 38.6542095	total: 1m 18s	remaining: 2.66s
14507:	learn: 38.6532200	total: 1m 18s	remaining: 

14659:	learn: 38.4918996	total: 1m 19s	remaining: 1.83s
14660:	learn: 38.4907944	total: 1m 19s	remaining: 1.83s
14661:	learn: 38.4894970	total: 1m 19s	remaining: 1.82s
14662:	learn: 38.4885345	total: 1m 19s	remaining: 1.82s
14663:	learn: 38.4880073	total: 1m 19s	remaining: 1.81s
14664:	learn: 38.4865948	total: 1m 19s	remaining: 1.81s
14665:	learn: 38.4850913	total: 1m 19s	remaining: 1.8s
14666:	learn: 38.4837923	total: 1m 19s	remaining: 1.79s
14667:	learn: 38.4830496	total: 1m 19s	remaining: 1.79s
14668:	learn: 38.4819189	total: 1m 19s	remaining: 1.78s
14669:	learn: 38.4808949	total: 1m 19s	remaining: 1.78s
14670:	learn: 38.4793986	total: 1m 19s	remaining: 1.77s
14671:	learn: 38.4784514	total: 1m 19s	remaining: 1.77s
14672:	learn: 38.4774207	total: 1m 19s	remaining: 1.76s
14673:	learn: 38.4761204	total: 1m 19s	remaining: 1.76s
14674:	learn: 38.4751429	total: 1m 19s	remaining: 1.75s
14675:	learn: 38.4741565	total: 1m 19s	remaining: 1.75s
14676:	learn: 38.4729687	total: 1m 19s	remaining:

14823:	learn: 38.3228337	total: 1m 19s	remaining: 949ms
14824:	learn: 38.3218866	total: 1m 19s	remaining: 943ms
14825:	learn: 38.3199724	total: 1m 19s	remaining: 938ms
14826:	learn: 38.3194082	total: 1m 19s	remaining: 933ms
14827:	learn: 38.3182512	total: 1m 19s	remaining: 927ms
14828:	learn: 38.3169166	total: 1m 19s	remaining: 922ms
14829:	learn: 38.3154366	total: 1m 19s	remaining: 916ms
14830:	learn: 38.3142886	total: 1m 19s	remaining: 911ms
14831:	learn: 38.3128487	total: 1m 19s	remaining: 906ms
14832:	learn: 38.3118142	total: 1m 19s	remaining: 900ms
14833:	learn: 38.3111191	total: 1m 19s	remaining: 895ms
14834:	learn: 38.3102666	total: 1m 19s	remaining: 889ms
14835:	learn: 38.3092861	total: 1m 19s	remaining: 884ms
14836:	learn: 38.3077190	total: 1m 19s	remaining: 879ms
14837:	learn: 38.3064160	total: 1m 19s	remaining: 873ms
14838:	learn: 38.3055918	total: 1m 19s	remaining: 868ms
14839:	learn: 38.3045193	total: 1m 20s	remaining: 863ms
14840:	learn: 38.3039572	total: 1m 20s	remaining

14986:	learn: 38.1529318	total: 1m 20s	remaining: 70.1ms
14987:	learn: 38.1513953	total: 1m 20s	remaining: 64.7ms
14988:	learn: 38.1510332	total: 1m 20s	remaining: 59.3ms
14989:	learn: 38.1495861	total: 1m 20s	remaining: 53.9ms
14990:	learn: 38.1490014	total: 1m 20s	remaining: 48.5ms
14991:	learn: 38.1483946	total: 1m 20s	remaining: 43.1ms
14992:	learn: 38.1475903	total: 1m 20s	remaining: 37.7ms
14993:	learn: 38.1463233	total: 1m 20s	remaining: 32.3ms
14994:	learn: 38.1454646	total: 1m 20s	remaining: 27ms
14995:	learn: 38.1436138	total: 1m 20s	remaining: 21.6ms
14996:	learn: 38.1423315	total: 1m 20s	remaining: 16.2ms
14997:	learn: 38.1415219	total: 1m 20s	remaining: 10.8ms
14998:	learn: 38.1398025	total: 1m 20s	remaining: 5.39ms
14999:	learn: 38.1384224	total: 1m 20s	remaining: 0us


In [125]:
#y_pred=model.predict(X_test)
#generate_test_files(y_pred)

In [82]:
#xgboostREG
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

param_grid = {'learning_rate':[1e-3, 1e-1], 
          'gamma':[0.3,0.4,0.5], 
           'min_child_weight':[4,5,6],
          'subsample':[0.06,0.08,0.09],
           'colsample_bytree':[0.06,0.07,0.08], 
           'max_depth': [2,3,4,5], 
           'n_estimators':[1000, 1500, 2000]}

xgb = xgboost.XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, param_grid)

In [83]:
grid.fit(X_train, y_train)
grid.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=None, colsample_bytree=0.6, gamma=0.3,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.1, max_delta_step=0, max_depth=4,
             min_child_weight=6, missing=None, monotone_constraints=None,
             n_estimators=1000, n_jobs=1, nthread=-1, num_parallel_tree=None,
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=True, subsample=1.0, tree_method=None,
             validate_parameters=None, verbosity=None)

In [121]:
y_pred = grid.best_estimator_.predict(X_test)
generate_test_files(y_pred)

___

# Summary
<a id="s"></a>


**After Grid Tuning the `XGBoost Regressor` model seems to have the best accuracy followed by `CATBoost`**
**Linear regression and RF Regressor had poor results**

*The final submission file is generated on the XGBoost model*


In [120]:
def generate_test_files(y_pred):
    submission = pd.DataFrame(y_pred, columns =['price']) 
    submission.to_csv("solution.csv",index=False)